In [1]:
%%html
<script type="text/javascript" src="/nbextensions/jupyter-jsmol/JSmol.min.js"></script>


In [2]:
# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Custom.html
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Low%20Level.html

from traitlets import Unicode, Bool, validate, TraitError, Dict
from ipywidgets import DOMWidget, register, Layout, Text, VBox, Button, HTML

@register
class JsmolView(DOMWidget):
    _view_name = Unicode('JsmolView').tag(sync=True)
    _view_module = Unicode('jsmol_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

    # Attributes
    info = Dict(help="The values for initialising the jmol applet").tag(sync=True)
    cmd = Unicode(help="The commands for jmol applet").tag(sync=True)

    def __init__(self, width="100%", height=400, color="gray", **kwargs):
        super().__init__(**kwargs)
        self.info = {
            'width': width,
            'height': height,
            'color': color,
            'use': "HTML5",
            'j2sPath': "/nbextensions/jupyter-dataviewer/j2s", 
            'antialiasDisplay': True,
            'disableInitialConsole': True,
            'disableJ2SLoadMonitor': True,
            'debug': False,
        }
            
    def script(self, cmd):
        # Note: so far only send command if it is differs from the last (only if there is any chages)
        # TODO: it should be only one directional
        self.cmd = cmd

    


In [3]:
%%javascript

require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min'
    }
});

<IPython.core.display.Javascript object>

In [4]:
%%javascript
equire('d3')

<IPython.core.display.Javascript object>

In [5]:
%%javascript
var jm = require('/nbextensions/jupyter-jsmol/JSmol.min.js');

<IPython.core.display.Javascript object>

In [6]:
%%javascript
// http://wiki.jmol.org/index.php/Jmol_JavaScript_Object/Functions

require.undef('jsmol_widget');

define('jsmol_widget', ["@jupyter-widgets/base"], function(widgets) {

    var JsmolView = widgets.DOMWidgetView.extend({
        
        // Render the view.
        render: function() {
            //JsmolView.__super__.render.apply(this, arguments);

            // every view should have its own unique id            
            this.el.setAttribute("id", widgets.uuid());

            // Create an Jmol applet
            this.create_app();
        
            // Python -> JavaScript update
            this.model.on('change:info', this.create_app, this);
            this.model.on('change:cmd', this.send_cmd, this);
            // this.listenTo(this.model, 'change:count', this._count_changed, this);


        },
        
        create_app: function() {
            console.log('Creating new object using the updated infos');
            console.log(this)

            // Note: jmol object needs to be created otherwise 
            // the callback function doesn't have access to info dict
            // jsmol_id should be a valid variable name for a js variable
            let jsmol_id = "jsmol_" + this.model.model_id

            Jmol.setDocument(false);
            this.jsmol = Jmol.getApplet(jsmol_id, this.model.get('info'));
            
            // Finally the the content of the div should be generated
            // Note: innerHTML method doesn't work for some reason...
            // this.jsmol_app.innerHTML = Jmol.getAppletHtml(jsmolapp_obj);
           
            var div_id = "#" + this.el.id;    // just a string
            var jsmol = this.jsmol;           // the object itself 
            
            $(div_id).ready(function() {
                $(div_id).html(Jmol.getAppletHtml(jsmol));
            });

        },
        
        send_cmd: function() {
            var script = this.model.get('cmd')
            this.model.set('cmd', '_');
            //this.model.save_changes();
            this.touch()
            
            
            Jmol.script(this.jsmol, script);
    
        }

    });

    return {
        JsmolView: JsmolView
    };
});



<IPython.core.display.Javascript object>

In [7]:
view1 = JsmolView(height=400, width='100%')
view1

JsmolView(info={'width': '100%', 'height': 400, 'color': 'gray', 'use': 'HTML5', 'j2sPath': '/nbextensions/jup…

In [8]:
view2 = JsmolView(height="100%", layout=Layout(height="600px"))
view2

JsmolView(info={'width': '100%', 'height': '100%', 'color': 'gray', 'use': 'HTML5', 'j2sPath': '/nbextensions/…

In [9]:
with open('example_structure.in') as f:
    data = f.read()
    
# print(data)

FileNotFoundError: [Errno 2] No such file or directory: 'example_structure.in'

In [10]:
# load inline "5 4 C 7.05 -5.74 C 8.27 -7.84 C 4.63 -7.14 C 5.84 -7.84 C 7.05 -7.14 1 5 1 2 5 1 3 4 1 4 5 1"

view1.script('load inline "' + data + '"')
view1.script('rotate on')

NameError: name 'data' is not defined

In [11]:
view2.script('load inline "' + data + '"')

NameError: name 'data' is not defined

In [12]:
view.widgets

NameError: name 'view' is not defined

In [13]:
view.close_all()

NameError: name 'view' is not defined

In [ ]:
view.script('load example_structure.in')
view.script('rotate on')

In [ ]:
view.script('load $CCC(C)C')

In [ ]:
view2 = JsmolView(layout=Layout(height="500px"))
view2

In [ ]:
view2.script('load example_structure.in')
view2.script('rotate off')

In [ ]:
from ase.io import read
from ase.visualize import view

In [ ]:
atoms = read('example_structure.in')
atoms
# view(atoms, viewer='ngl')

In [ ]:
from ase.io import write


In [ ]:
?write

In [ ]:
import io

with io.StringIO() as output:
    write(output, atoms, format='aims', parallel=False)
    data = output.getvalue()
data

In [ ]:
?write

In [ ]:
data

In [ ]:
%%javascript
// https://chemapps.stolaf.edu/jmol/docs/

Jmol.script(jsmolapp_div, 'load example_structure.in')
Jmol.script(jsmolapp_div, 'rotate off')



In [ ]:
%%javascript
Jmol.script(jsmolapp_div, 'set showUnitcell false')
Jmol.script(jsmolapp_div, 'set showUnitcellDetails false')

In [ ]:
view.test='sdfsf'

In [ ]:
view.info=dict(view.info)

In [ ]:
%%html
<script type="text/javascript">
$("#jsmolapp_div").ready(function() {
    Info = {
        width: "100%",
        height: "100%",
        color: "gray",
        use: "HTML5",
        j2sPath: "/nbextensions/jupyter-dataviewer/j2s",
        antialiasDisplay: true,
        disableInitialConsole: true,
        disableJ2SLoadMonitor: true
    };
    $("#jsmolapp_div").html(Jmol.getAppletHtml("jsmolapp_obj2", Info));
});
</script>

In [ ]:
%%html
<script type="text/javascript">
$("#jsmolapp_div").ready(function() {
    Info = {
        width: "100%",
        height: "100%",
        color: "blue",
        script: "load example_structure.in;",
        use: "HTML5",
        j2sPath: "/nbextensions/jupyter-dataviewer/j2s",
        antialiasDisplay: true,
        disableInitialConsole: true,
        disableJ2SLoadMonitor: true,
    },
    $("#jsmolapp_div").html(Jmol.getAppletHtml("jsmolapp", Info));
});
</script>

In [ ]:
// %%javascript

// require.undef('jsmol_widget');

// define('jsmol_widget', ["@jupyter-widgets/base"], function(widgets) {

//     var JsmolView = widgets.DOMWidgetView.extend({

//         // Render the view.
//         render: function() {
            
//             this.el.setAttribute("id", "jsmolapp_div");
//             console.log(this);
//             console.log(widgets);

//             // Creating placeholder for Jsmol
//             // this.jsmol_app = document.createElement('div');
//             // this.jsmol_app.setAttribute("id", "jsmolapp_div");
//             // this.jsmol_app.style.height = "100%";
//             // this.el.appendChild(this.jsmol_app);

//             this.create_app()

//             // Python -> JavaScript update
//             this.model.on('change:info', this.create_app, this);

//         },

//         create_app: function() {
//             console.log('Creating new object using the updated infos');

//             // Create an Jmol applet
//             // Note: jmol object needs to be created otherwise 
//             // the callback function doesn't have access to info dict
//             Jmol.setDocument(false);
//             Jmol.getApplet("jsmolapp_obj", this.model.get('info'));

//             // Finally the the content of the div should be generated
//             // Note: innerHTML method doesn't work for some reason...
//             // this.jsmol_app.innerHTML = Jmol.getAppletHtml(jsmolapp_obj);
//             $("#jsmolapp_div").ready(function() {
//                 $("#jsmolapp_div").html(Jmol.getAppletHtml(jsmolapp_obj));
//             });

//         },

//         info_changed: function() {
//             console.log(this.model.get('info'));
//             //$("#jsmolapp").html(Jmol.getAppletHtml(jsmolapp_obj));
//             //this.jsmol_app.innerHTML = Jmol.getAppletHtml(jsmolapp_obj);
//             console.log(this.el)
//         },

//     });

//     return {
//         JsmolView: JsmolView
//     };
// });

// // // http://wiki.jmol.org/index.php/Jmol_JavaScript_Object/Functions